### Authentication and Authorization System

1. OAuth Grant type: Authorization Code Grant<br>
2. Support for multi factor authorization<br>
3. JWT Token service

###Roadmap to follow
1. Creating a user authentication system
2. Store client application and permission
3. Scope based access control can be chosen (read, write, etc.)
4. Generate authorization codes & access tokens when users grant permissions

### Creating and Testing Database

In [ ]:
# testing database SQLite
import sqlite3 as sq

conn = sq.connect("test_database.db")
cursor = conn.cursor()

# Creating a table
cursor.execute("CREATE TABLE IF NOT EXISTS users (id INTEGER PRIMARY KEY, name TEXT)")

# Inserting values into the table
cursor.execute("INSERT INTO users (name) VALUES ('Alice')")
conn.commit()

# fetching the values
cursor.execute("SELECT * FROM users")
print(cursor.fetchall())

cursor.close()
conn.close()

# Testing worked perfectly ✔

[(1, 'Alice')]


In [86]:
import getpass

In [87]:
# Creating the database for the storing the values
import sqlite3 as sq
conn = sq.connect("user_credentials.db")
cursor = conn.cursor()

# Enable foreign key support
conn.execute("PRAGMA foreign_keys = ON;")

# Creating the table
# username(unique), firstname, lastname, email, password
cursor.execute("""
  CREATE TABLE IF NOT EXISTS user_credentials(
    username varchar(20) primary key,
    firstname varchar(20),
    lastname varchar(20),
    email varchar(40),
    password_hash varchar(10),
    FOREIGN KEY (password_hash) REFERENCES password(password_hash) ON DELETE CASCADE ON UPDATE CASCADE
  )
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS password(
    password_hash varchar(10) PRIMARY KEY,
    password varchar(100)
  )
""")

cursor.close()
conn.close()

### Sign Up Feature

In [88]:
# Sign up feature
# Storing the relevant information
userName = input("Create a username: ")
firstName = input("Enter your First Name: ")
lastName = input("Enter your Last Name: ")
email = input("Enter your E-mail: ")
password = getpass.getpass("Create Password: ")
confirmPassword = getpass.getpass("Enter the previously entered password: ")

# verify if email is valid or not?
def verify_email(email):
  tld = [".com", ".org", ".net", ".in", ".org"]
  if email[-4:] in tld and "@" in email:
    if ".." in email:
      return False
    else:
      return True
  else:
    return False

# verify if the password entered is correct or not
def check_password(password, confirmPassword):
  return password==confirmPassword

Create a username: mrityunjay2505
Enter your First Name: Mrityunjay
Enter your Last Name: Shukla
Enter your E-mail: mrityunjay.shukla@bt.com
Create Password: ··········
Enter the previously entered password: ··········


In [89]:
print(verify_email(email))

True


Creating and testing a hashing technique to implement password security
It should have<br>
✅ **Irreversibility**: Cannot recover the original password from the hash.<br>
✅ **Avalanche Effect**: Small changes in input drastically change the hash.<br>
✅ **Salting**: A random value is added to prevent hash collisions.<br>
✅ **High Entropy**: Harder to guess using brute force.

In [90]:

# Creating the hashing function to encrypt the password
def password_encryption(password, salt="random_value"):
  hashed_value=0
  input_data = password + salt
  for char in input_data:
    hashed_value = (hashed_value * 31 + ord(char)) % (2**32)
  return hex(hashed_value)


In [91]:
# Get the hash for the password
if check_password(password, confirmPassword):
  hash_password = password_encryption(password)


In [92]:
# inserting the signup values
conn = sq.connect("user_credentials.db")
cursor = conn.cursor()

# check the email and password, if correct then enter in the db
# else return the error statement
if verify_email(email) and check_password(password, confirmPassword):
  cursor.execute(f"INSERT INTO user_credentials VALUES('{userName}','{firstName}','{lastName}','{email}','{hash_password}')")
  conn.commit()

  cursor.execute(f"INSERT INTO password VALUES('{hash_password}', '{password}')")
  conn.commit()
else:
  print("Error in the storing")
  print("Either the email is invalid or the passwords don't match")

cursor.close()
conn.close()

# Note: Create a situation where if username exists, doesn't returns error

In [93]:
# Get the value stored
conn = sq.connect("user_credentials.db")
cursor = conn.cursor()

# Fetch the values
cursor.execute("SELECT * FROM user_credentials")
print(cursor.fetchall())

cursor.execute("SELECT * FROM password")
print(cursor.fetchall())

cursor.close()
conn.close()

[('mrityunjay2505', 'Mrityunjay', 'Shukla', 'mrityunjay.shukla@bt.com', '0x44ae9af8')]
[('0x44ae9af8', 'Manu2505')]


In [95]:
# Get the credentials and password
conn = sq.connect("user_credentials.db")
cursor = conn.cursor()

cursor.execute("""
  SELECT u.username, u.firstname, u.lastname, u.email, p.password
  FROM user_credentials as u JOIN password as p
  ON u.password_hash=p.password_hash
""")
entries = cursor.fetchall()

for entry in entries:
  print(entry)

('mrityunjay2505', 'Mrityunjay', 'Shukla', 'mrityunjay.shukla@bt.com', 'Manu2505')


In [77]:
# Delete a record (TESTING)
conn = sq.connect("user_credentials.db")
cursor = conn.cursor()

cursor.execute("DELETE FROM user_credentials where username='mrityunjay2505'")
conn.commit()


cursor.close()
conn.close()

In [85]:
# Delete a record
conn = sq.connect("user_credentials.db")
cursor = conn.cursor()

cursor.execute("DROP TABLE user_credentials")
conn.commit()

cursor.execute("DROP TABLE password")
conn.commit()

### Log in Feature

In [100]:
# Log in feature
# The log in feature will check for the username
# and for that username, see if the email and password connected are matching or not
loginUserName = input("Enter your username: ")

conn = sq.connect("user_credentials.db")
cursor = conn.cursor()

# cursor.execute("SELECT * FROM user_credentials WHERE username = ?", (loginUserName,))
cursor.execute("""
  SELECT u.username, u.firstname, u.lastname, u.email, p.password
  FROM user_credentials as u JOIN password as p
  ON u.password_hash=p.password_hash
""")
entry = cursor.fetchone()

entry_list = list(entry) if entry else []

cursor.close()
conn.close()

def entry_valid(entry_list):
  return True if len(entry_list) == 5 else False

loginPassword = input("Enter password: ")

def check_login_password(password, storedPassword):
  return password == storedPassword

if entry_valid(entry_list) and loginUserName==entry_list[0]:
  loginUserName = entry_list[0]
  name = entry_list[1] + entry_list[2]
  if check_login_password(loginPassword, entry_list[4]):
    loginPassword = entry_list[4]
  else:
    print("Error in the password")
    loginUserName = ""
    name = ""
    loginPassword = ""
    loginEmail = ""

  loginEmail = entry_list[3]
  print(loginEmail, loginPassword)
else:
  print("Error: User doesn't exist")
  loginUserName = ""
  name = ""
  loginPassword = ""
  loginEmail = ""


Enter your username: mrityunjay2505
Enter password: Manu2505
mrityunjay.shukla@bt.com Manu2505
